In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
!pip3 install omegaconf
!pip3 install einops
!pip3 install pytorch-lightning==1.7.1
!pip3 install torchmetrics==0.11.4
!pip3 install taming-transformers
!pip3 install kornia
!pip3 install git+https://github.com/openai/CLIP.git
!pip3 install openai-clip

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-3qyavpow
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-3qyavpow
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done


In [9]:
cd /content/drive/MyDrive/CIS581/InST

[Errno 2] No such file or directory: '/content/drive/MyDrive/CIS581/InST'
/content


In [10]:
from evaluation.clip_eval import CLIPEvaluator
import torch
import torch.nn as nn
import numpy as np
import clip
from PIL import Image
import PIL

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
clipEvaluator = CLIPEvaluator(device)

def load_img(path):
    image = Image.open(path).convert("RGB")
    w, h = image.size
    print(f"loaded input image of size ({w}, {h}) from {path}")
    w, h = map(lambda x: x - x % 32, (w, h))  # resize to integer multiple of 32
    image = image.resize((512, 512), resample=PIL.Image.LANCZOS)
    image = np.array(image).astype(np.float32) / 255.0
    image = image[None].transpose(0, 3, 1, 2)
    image = torch.from_numpy(image)
    return 2.*image - 1.

ModuleNotFoundError: ignored

In [ ]:
style = load_img('data/andre.jpg')
content = load_img('data/IMG_1581.JPG')
style2 = load_img('data/monet.jpg')
generated = load_img('outputs/img2img-samples/IMG_1581-*-0050.png')
i2i = clipEvaluator.img_to_img_similarity(style2, generated)

print(i2i)

In [ ]:
# text = 'a modern style of painting of the city'
# content = load_img('data/IMG_1581.JPG')
# generated = load_img('outputs/img2img-samples/IMG_1581-*-0050.png')
# t2i = clipEvaluator.txt_to_img_similarity(text, content)

# print(t2i)

In [ ]:
import numpy as np
from numpy import cov
from numpy import trace
from numpy import iscomplexobj
from numpy import asarray
from numpy.random import randint
from scipy.linalg import sqrtm
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.datasets.mnist import load_data
from skimage.transform import resize
import tensorflow as tf
from math import sqrt
import cv2

input_shape = (256, 256, 3)

# def scale_images(image, new_shape):
#   images_list = []
#   new_image = resize(image, new_shape, 0)
#   images_list.append(new_image)
#   return asarray(images_list)

def calculate_fid(model, images1, images2, new_shape=input_shape): #lower is better
    images1_np = images1.permute(0, 2, 3, 1).cpu().numpy()
    images2_np = images2.permute(0, 2, 3, 1).cpu().numpy()
    images1_np = np.array([cv2.resize(img, (256, 256)) for img in images1_np])
    images2_np = np.array([cv2.resize(img, (256, 256)) for img in images2_np])
    images1_np = preprocess_input(images1_np)
    images2_np = preprocess_input(images2_np)

    act1 = model.predict(images1_np)
    act2 = model.predict(images2_np)
    mu1, sigma1 = act1.mean(axis=0), np.cov(act1, rowvar=False)
    mu2, sigma2 = act2.mean(axis=0), np.cov(act2, rowvar=False)
    ssdiff = np.sum((mu1 - mu2) ** 2.0)

    covmean = sqrt(sigma1*sigma2)
    if np.iscomplexobj(covmean):
        covmean = covmean.real

    fid = ssdiff + sigma1 + sigma2 - 2.0 * covmean
    return fid

IncModel = InceptionV3(include_top=False, pooling='avg', input_shape=input_shape)


In [ ]:
images1 = load_img('data/andre.jpg')
monet = load_img('data/monet.jpg')
modern = load_img('data/modern.jpg')
content = load_img('data/IMG_1581.JPG')

images2 = load_img('outputs/img2img-samples/IMG_1581-*-0051.png')
images3 = load_img('outputs/img2img-samples/IMG_1581-*-0050.png')
fid_value = calculate_fid(IncModel, style, images3) #only style
print(fid_value)

In [11]:
import cv2
from google.colab.patches import cv2_imshow

content_image = cv2.imread('data/IMG_1581.JPG')
generated_image = cv2.imread('outputs/img2img-samples/IMG_1581-*-0050.png')

image = cv2.resize(content_image, (512, 512))
img_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

ret, thresh = cv2.threshold(img_gray, 150, 255, cv2.THRESH_BINARY)
cv2_imshow(thresh)
cv2.waitKey(0)
cv2.imwrite('image_thres1.jpg', thresh)
cv2.destroyAllWindows()

# contours, hierarchy = cv2.findContours(image=thresh, mode=cv2.RETR_TREE, method=cv2.CHAIN_APPROX_NONE)

# # draw contours on the original image
# image_copy = image.copy()
# cv2.drawContours(image=image_copy, contours=contours, contourIdx=-1, color=(0, 255, 0), thickness=2, lineType=cv2.LINE_AA)

# # see the results
# cv2_imshow(image_copy)
# cv2.waitKey(0)
# cv2.imwrite('contours_none_image1.jpg', image_copy)
# cv2.destroyAllWindows()

contours1, hierarchy1 = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
image_copy1 = image.copy()
cv2.drawContours(image_copy1, contours1, -1, (0, 255, 0), 2, cv2.LINE_AA)
cv2_imshow(image_copy1)
cv2.waitKey(0)
cv2.imwrite('contours_simple_image1.jpg', image_copy1)
cv2.destroyAllWindows()


image1 = cv2.resize(generated_image, (512, 512))
img_gray = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)

ret, thresh = cv2.threshold(img_gray, 150, 255, cv2.THRESH_BINARY)
cv2_imshow(thresh)
cv2.waitKey(0)
cv2.imwrite('image_thres1.jpg', thresh)
cv2.destroyAllWindows()

contours2, hierarchy2 = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
image_copy2 = image1.copy()
cv2.drawContours(image_copy2, contours2, -1, (0, 255, 0), 2, cv2.LINE_AA)
cv2_imshow(image_copy2)
cv2.waitKey(0)
cv2.imwrite('contours_simple_image1.jpg', image_copy2)
cv2.destroyAllWindows()

print(len(contours1), len(contours2))



error: ignored

In [ ]:
# ----- none -----

import cv2
import numpy as np

def calculate_iou(contour1, contour2):
    # Create binary images from the contours
    img1 = np.zeros((512, 512), dtype=np.uint8)
    img2 = np.zeros((512, 512), dtype=np.uint8)
    cv2.drawContours(img1, [contour1], -1, 1, -1)
    cv2.drawContours(img2, [contour2], -1, 1, -1)

    # Calculate intersection and union
    intersection = np.logical_and(img1, img2)
    union = np.logical_or(img1, img2)
    iou = np.sum(intersection) / np.sum(union)
    return iou

def evaluate_contours(true_contours, detected_contours, iou_threshold=0.5):
    TP, FP, FN = 0, 0, 0

    matched = set()
    for d_contour in detected_contours:
        is_match = False
        for idx, t_contour in enumerate(true_contours):
            if calculate_iou(d_contour, t_contour) > iou_threshold and idx not in matched:
                TP += 1
                matched.add(idx)
                is_match = True
                break
        if not is_match:
            FP += 1

    FN = len(true_contours) - len(matched)

    precision = TP / (TP + FP) if TP + FP > 0 else 0
    recall = TP / (TP + FN) if TP + FN > 0 else 0
    f_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return precision, recall, f_score


precision, recall, f_score = evaluate_contours(contours2, contours1)
print(f"Precision: {precision:.3f}")
print(f"Recall: {recall:.3f}")
print(f"F-score: {f_score:.3f}")

In [ ]:
def calc_precision_recall(contours_a, contours_b, threshold):
    x = contours_a
    y = contours_b

    xx = np.array(x)
    hits = []
    for yrec in y:
        d = np.square(xx[:,0] - yrec[0]) + np.square(xx[:,1] - yrec[1])
        hits.append(np.any(d < threshold*threshold))
    top_count = np.sum(hits)

    try:
        precision_recall = top_count / len(y)
    except ZeroDivisionError:
        precision_recall = 0

    return precision_recall, top_count, len(y)


precision_recall, top_count, length = calc_precision_recall(contours2, contours1,0.5)
print(f"Precision: {precision_recall:.3f}")
print(f"Recall: {top_count:.3f}")
print(f"F-score: {length:.3f}")

In [ ]:

image1 = cv2.imread('data/modern.jpg')
image1 = cv2.resize(image1, (512, 512))

img_gray1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)

ret, thresh1 = cv2.threshold(img_gray1, 150, 255, cv2.THRESH_BINARY)
contours2, hierarchy2 = cv2.findContours(thresh1, cv2.RETR_TREE,
                                               cv2.CHAIN_APPROX_SIMPLE)
image_copy2 = image1.copy()
cv2.drawContours(image_copy2, contours2, -1, (0, 255, 0), 2, cv2.LINE_AA)
cv2_imshow(image_copy2)
cv2.waitKey(0)
image_copy3 = image1.copy()
for i, contour in enumerate(contours2): # loop over one contour area
   for j, contour_point in enumerate(contour): # loop over the points
       # draw a circle on the current contour coordinate
       cv2.circle(image_copy3, ((contour_point[0][0], contour_point[0][1])), 2, (0, 255, 0), 2, cv2.LINE_AA)
# see the results
cv2_imshow(image_copy3)
cv2.waitKey(0)
cv2.imwrite('contour_point_simple.jpg', image_copy3)
cv2.destroyAllWindows()
print(len(contours2))

In [ ]:
from numpy import asarray
from numpy import expand_dims
from numpy import log
from numpy import mean
from numpy import exp

# calculate the inception score for p(y|x)
def calculate_inception_score(p_yx, eps=1E-16):
    p_y = expand_dims(p_yx.mean(axis=0), 0)
    kl_d = p_yx * (log(p_yx + eps) - log(p_y + eps))
    sum_kl_d = kl_d.sum(axis=1)
    avg_kl_d = mean(sum_kl_d)
    is_score = exp(avg_kl_d)
    return is_score